In [2]:
import torch
from torch import nn

from torchsummary import summary

In [3]:
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# output = loss(input, target)
# output.backward()

In [4]:
loss = nn.CrossEntropyLoss()

input = torch.randn(3, 5, requires_grad=True)

target = torch.empty(3, dtype=torch.long).random_(5)

output = loss(input, target)

output.backward()

In [5]:
print(input)

tensor([[ 1.8919, -1.2054,  0.5891, -1.8915, -0.3432],
        [-1.5370,  0.3028, -0.4971, -0.4308,  1.2327],
        [ 1.1624,  0.9201,  0.8554, -0.1789,  1.4063]])


In [6]:
print(target)

tensor([ 1,  1,  2])


In [7]:
from torchvision import transforms, utils, models

In [53]:
a = models.resnet34(pretrained=True)

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = a.to(device)

summary(a, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [55]:
a

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [59]:
list(a.children())[-2]

AvgPool2d(kernel_size=7, stride=1, padding=0)

In [10]:
b = models.resnet34(pretrained=True)

In [49]:
list(b.children())[-1]

Linear(in_features=25088, out_features=4, bias=True)

In [12]:
removed = list(b.children())[:-2]

In [13]:
resnet34_head = nn.Sequential(*removed)

In [14]:
resnet34_head.to(device)
summary(resnet34_head, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [38]:
class Flatten(nn.Module):
    """flattern layer"""
    def forward(self, x):
        return x.view(x.size()[0], -1)

class RNetCustom(nn.Module):
    def __init__(self):
        super().__init__()
        self.num_bbox_param = 4 
        
        self.backbone = self._prep_backbone()
        self.flatten = Flatten()
        self.fc = nn.Linear(512 * 7 * 7, self.num_bbox_param)
            
    def _prep_backbone(self):     
        base_model = models.resnet34(pretrained=True)
        removed = list(base_model.children())[:-3]
        backbone = nn.Sequential(*removed)
        for param in backbone.parameters():
            param.require_grad = False
        return backbone
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.flatten(x)
        y_pred = self.fc(x)
        return y_pred

In [39]:
b = RNetCustom()

b.to(device)
summary(b, (3, 224, 224))

RuntimeError: size mismatch, m1: [2 x 50176], m2: [25088 x 4] at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMathBlas.cu:249

In [17]:
b = [1,2,3]

In [18]:
b = torch.tensor(b)

In [19]:
b.type()

'torch.LongTensor'

In [20]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()

In [21]:
loss = nn.MSELoss()

In [22]:
input = torch.randn(3, 5, requires_grad=True)

In [23]:
input

tensor([[-1.1447, -1.9161,  0.0695,  1.5623,  0.4595],
        [-1.0395,  0.4035, -0.2688, -0.3646,  1.0763],
        [-1.6512,  0.7550,  2.1010,  0.0662,  1.4468]])

In [24]:
target = torch.randn(3, 5)
target

tensor([[ 0.3530,  0.3034, -0.0193, -0.6803, -0.3755],
        [-0.8435, -1.9644, -0.1167, -0.5139,  0.3826],
        [-0.4257, -0.3014, -0.5506,  0.8362, -1.0045]])

In [25]:
output = loss(input, target)

In [26]:
output

tensor(2.3551)

In [27]:
output.backward()

In [28]:
a = [1,2,3,4,5]
b = [i for i in a if i > 10]

In [29]:
b

[]

In [30]:
input = torch.randn((3, 3, 224, 224), requires_grad=True)

In [31]:
input = input.to(device)

In [32]:
b = RNetCustom()
b = b.to(device)

In [33]:
b(input)

tensor([[ 1.2530,  1.0162,  1.4152,  0.3180],
        [-0.0612, -1.2376,  0.0151, -0.6957],
        [ 0.4807, -0.8007,  0.7537,  0.4913]], device='cuda:0')